## Regressions


In [42]:
## load packages

import pandas as pd
import datetime

#SKLEARN
from sklearn.model_selection import train_test_split

# for vectorization 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#For word handeling
import string 
import nltk
nltk.download('punkt') # you will probably need to do this
nltk.download('wordnet') # and this
nltk.download('stopwords') # aand this

# for classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


import numpy as np


import time

[nltk_data] Downloading package punkt to C:\Users\Liv
[nltk_data]     Nøhr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Liv
[nltk_data]     Nøhr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Liv
[nltk_data]     Nøhr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocess(text):
    low_text= text.lower()
    low_text = low_text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(low_text)
    porter = nltk.WordNetLemmatizer()
    lemmatizer=[porter.lemmatize(t) for t in tokens]
    stop_words_list = stoppelop
    sent_sw_removed = [i for i in lemmatizer if i not in stop_words_list]
    lemmas=[i for i in sent_sw_removed if i!='br']
    return lemmas # return a list of stems/lemmas

In [26]:
## Get data
df_c = pd.read_csv('df_comments_final.csv')
df_c = df_c.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_c=df_c[df_c['Comments']!='No Comments']

df_c['year'] = df_c['Dates'].str[2:4]

#Divide into target and features
y= df_c['status']

X = df_c['Comments']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=161193)

print(X_train)

70307    She is rude and she hardly really taught the c...
32222    i am in class right now..i would honestly rath...
41925    Dr. Jack Thacker is the best professor you wil...
41720    Multiple Tegrity lecture recordings that were ...
32150                                Avoid this professor.
                               ...                        
47433    He is a great teacher and cares for his studen...
55647    Professor Kan does not give exams other than t...
40374    Rich makes you work, but was really helpful ou...
18837    Dr. Wang  was well prepared for class, and tau...
2852                   Not very friendly for office hours.
Name: Comments, Length: 57654, dtype: object


In [31]:
with open('STOPlist.txt') as f:
    stoppelop = f.read().splitlines()
stoppelop


['cal',
 'calu',
 'jhu',
 'mit',
 'orgo',
 'pwr',
 'uchicago',
 'cussw',
 'ssa',
 'csu',
 'wsu',
 'wku',
 'twu',
 'pba',
 'ut',
 'utt',
 'uhh',
 'asl',
 'nan',
 'gamm',
 'york',
 'berlin',
 'ray',
 'pat',
 'simpser',
 'beck',
 'toscano',
 'ochs',
 'rodriguezjimenez',
 'pruitt',
 'shaunna',
 'kirven',
 'susannah',
 'robbins',
 'ryan',
 'shakti',
 'ronchen',
 'neenan',
 'bob',
 'mcelvy',
 'salari',
 'malkin',
 'ebach',
 'samuel',
 'lava',
 'guerrero',
 'leanna',
 'linehan',
 'merenda',
 'sanaz',
 'rorabaugh',
 'neely',
 'shannan',
 'carfango',
 'embertontinus',
 'benshahar',
 'noam',
 'atkinson',
 'gorski',
 'nahee',
 'liisa',
 'lasley',
 'vega',
 'keat',
 'hallie',
 'madsen',
 'bukonda',
 'andrew',
 'ze',
 'joonhyun',
 'hoffmann',
 'laurentsimpson',
 'mccabe',
 'wei',
 'planning',
 'public',
 'policy',
 'janis',
 'chertow',
 'mabel',
 'karem',
 'rundell',
 'noble',
 'willome',
 'brittnee',
 'movassaghi',
 'creasy',
 'juneau',
 'preslan',
 'grice',
 'sijie',
 'hebron',
 'sang',
 'golnaz'

In [37]:
#Vectorize - both using count and tfidf

start = time.time()
#using count and own preprocesser
vectorizerc = CountVectorizer(tokenizer=preprocess)

# The top N most frequent features:
#vectorizerc_most = CountVectorizer(max_features=80, tokenizer=preprocess)

#using tfidf vectorizer
vectorizertfidf = TfidfVectorizer(tokenizer=preprocess)


#Using tfidf vectorizer w highest features
#vectorizertfidf = Tfidf_mostVectorizer(tokenizer=preprocess)

# fit and transform train and test set for each vectorizer:
X_train_c = vectorizerc.fit_transform(X_train)
#X_train_c_most = vectorizerc_most.fit_transform(X_train)
X_train_tf = vectorizertfidf.fit_transform(X_train)


# Only tranform test set: never fit your vectorizer on the test set (it is cheating). Out-of-Vocabulary words are handled automatically be sklearn's vectorizer.
X_test_c = vectorizerc.transform(X_test)
#X_test_bow = vectorizerc_most.transform(X_test)
X_test_tf = vectorizertfidf.transform(X_test)

end = time.time()
print(end-start)


1225.5741655826569


### Check difference in precision of count and tf_idf vectorization (using lasso)

In [ ]:
start = time.time()

# classifier
lr = LogisticRegression(random_state=0, penalty = 'l1', solver = 'saga', max_iter=4000)

#training
lr.fit(X_train_c,y_train)

#testing
train_preds = lr.predict(X_train_c)
test_preds = lr.predict(X_test_c)
print("training accuracy:", np.mean([(train_preds==y_train)]))
print("testing accuracy:", np.mean([(test_preds==y_test)]))

end = time.time()

In [44]:
# classifier
lr = LogisticRegression(random_state=0, penalty = 'l1', solver = 'saga')

#training
lr.fit(X_train_tf,y_train)

#testing
train_preds = lr.predict(X_train_tf)
test_preds = lr.predict(X_test_tf)
print("training accuracy:", np.mean([(train_preds==y_train)]))
print("testing accuracy:", np.mean([(test_preds==y_test)]))

training accuracy: 0.8096923023554307
testing accuracy: 0.7964408367155792


### Check difference between Lasso and Ridge

In [ ]:
start = time.time()

# classifier
lr = LogisticRegression(random_state=0, penalty = 'l2', solver = 'saga', max_iter=4000)

#training
lr.fit(X_train_c,y_train)

#testing
train_preds = lr.predict(X_train_c)
test_preds = lr.predict(X_test_c)
print("training accuracy:", np.mean([(train_preds==y_train)]))
print("testing accuracy:", np.mean([(test_preds==y_test)]))

end = time.time()